In [1]:
import mysql.connector
import pandas as pd

# **Recaudo**

In [2]:
try:
    conexion = mysql.connector.connect(
        user="root",
        password='acr*162',
        host='192.168.192.118',
        database='acrjava',
        port='3306'
    )

    if conexion.is_connected():
        cursor = conexion.cursor()
        sql = """
        SELECT 
            t.CN608TRXNREFNO AS REFERENCIA_PAGO,
            DATE_FORMAT(t.CN608TRXNQSYSDATE, '%d/%m/%Y') AS FECHA_PAGO,
            CONCAT(RTRIM(c.cn401clientname1), RTRIM(c.cn401clientname2)) AS FRANQUICIA, 
            s.CN662SHOPNAME AS TIENDA, 
            CASE 
                WHEN s.CN662SHOPNAME LIKE '%PSE%' AND s.CN662SHOPNAME NOT LIKE '%PAGOS PSE AVAL%' THEN 'PAGO PSE IRIS'
                WHEN s.CN662SHOPNAME LIKE '%PAGOS PSE AVAL%' THEN 'PAGOS PSE AVAL'
                WHEN s.CN662SHOPNAME IN ('INTERGESTIONA S.A.S', 'BINANCE PAY', 'DAVIPLATA', 'CORRESPONSAL') THEN s.CN662SHOPNAME
                WHEN s.CN662SHOPNAME IN ('OFICINA PRINCIPAL ACR PLUS', 'CHEQUES Y TRANSFER BOGOTA', 'DESCUENTO DE PROVEEDORES', 'DESCUENTOS DE EMPLEADOS') THEN 'OFICINA PRINCIPAL ACR PLUS'
                ELSE 'COMERCIOS ALIADOS' 
            END AS CATEGORIA_TIENDA,
            t.CN608CLIENTIDNO AS IDENTIFICACION,
            t.CN608TRXNAMOUNT/100 AS PAGO
        FROM 
            ps608cardtrxn t
        INNER JOIN 
            ps662shopdetails s ON t.CN608TRXNSHOPCODE = s.CN662SHOPCODE
        INNER JOIN 
            ps401client c ON s.CN662SHOPPCODE = c.CN401CLIENTNO
        WHERE 
            t.CN608TRXNTYPE = 'P'
            AND t.CN608TRXNSTATUS = 'P'
            AND t.CN608TRXNQSYSDATE BETWEEN '20240701' AND '20240721'
            AND s.CN662SHOPNAME NOT LIKE '%ajuste%'
            AND s.CN662SHOPNAME NOT LIKE '%ajustes%'
            AND s.CN662SHOPNAME NOT LIKE '%rediferido%'
        """
        cursor.execute(sql)
        
        # Obtener todas las filas de la consulta ejecutada
        resultados = cursor.fetchall()
        
        # Toma los nombres de las columnas
        columnas = [desc[0] for desc in cursor.description]
        
        # Crea dataframe
        df = pd.DataFrame(resultados, columns=columnas)

except mysql.connector.Error as err:
    print(f'Error: {err}')

finally:
    if 'conexion' in locals() and conexion.is_connected():
        cursor.close()
        conexion.close()
        print('Conexión cerrada')
        
#df.to_csv('./files/recaudos.csv', index=False)

Conexión cerrada


In [3]:
df

,REFERENCIA_PAGO,FECHA_PAGO,FRANQUICIA,TIENDA,CATEGORIA_TIENDA,IDENTIFICACION,PAGO
0,240702-8924,02/07/2024,BERNAL FRANCO DIEGO,SPORTWEAR,COMERCIOS ALIADOS,66654461,30000.00
1,240702-8925,02/07/2024,BERNAL FRANCO DIEGO,SPORTWEAR,COMERCIOS ALIADOS,29533378,38000.00
2,240702-8926,02/07/2024,BERNAL FRANCO DIEGO,SPORTWEAR,COMERCIOS ALIADOS,24244520,164700.00
3,240703-8928,03/07/2024,BERNAL FRANCO DIEGO,SPORTWEAR,COMERCIOS ALIADOS,16860352,152933.93
4,240703-8929,03/07/2024,BERNAL FRANCO DIEGO,SPORTWEAR,COMERCIOS ALIADOS,1114836840,47308.81
...,...,...,...,...,...,...,...
8350,240704-52,04/07/2024,CARTAGENA SVL S.A.S,CARTAGENA SVL S.A.S,COMERCIOS ALIADOS,31894538,96704.32
8351,240704-54,04/07/2024,CARTAGENA SVL S.A.S,CARTAGENA SVL S.A.S,COMERCIOS ALIADOS,1127925571,155288.09
8352,240708-60,08/07/2024,CARTAGENA SVL S.A.S,CARTAGENA SVL S.A.S,COMERCIOS ALIADOS,29683643,83352.13
8353,240716-67,16/07/2024,CARTAGENA SVL S.A.S,CARTAGENA SVL S.A.S,COMERCIOS ALIADOS,1114239688,106485.00


In [4]:
df = pd.read_csv('../datos/recaudos.csv', sep=';')

In [14]:
df

,REFERENCIA_PAGO,FECHA_PAGO,FRANQUICIA,TIENDA,CATEGORIA_TIENDA,IDENTIFICACION,PAGO
